In [4]:
import datetime
from ib_insync import *
import pandas as pd
import numpy as np
import datetime as dt
import ib_insync.util as util2
util2.startLoop()

ib = IB()
ib.connect()

sma_s = 2
sma_l = 5
freq = '1 min'
units = 1000
end_time = dt.time(3,40, 0)
contract = Forex("EURUSD")
ib.qualifyContracts(contract)
cfd = CFD('EUR', currency='USD')
ib.qualifyContracts(cfd)
conID = cfd.conId


In [7]:
def on_bar_update(bars, hasNewBar):
    global df, last_bar_date, df_strategy

    if True:
        last_bar_date = bars[-1].date

        #data processing
        df = pd.DataFrame(bars)[['date', 'open', 'high', 'low', 'close']].iloc[:-1]
        df.set_index('date', inplace=True)

        ##### TRADING STRATEGY
        df_strategy = df[['close']].copy()
        df_strategy['sma_s'] = df_strategy.close.rolling(sma_s).mean()
        df_strategy['sma_l'] = df_strategy.close.rolling(sma_l).mean()
        df_strategy.dropna(inplace=True)
        df_strategy['position'] = np.where(df_strategy['sma_s'] > df_strategy['sma_l'], 1, -1)
        ######################

        #target = df_strategy['position'][-1] * units

        target = df_strategy['position'].iloc[-1] * units

        print('iloc:')
        print(df_strategy['position'].iloc[-1])

    #    print(df)
    else:
        pass

In [8]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True
    )

bars.updateEvent += on_bar_update

In [11]:
df_strategy['position']

date
2024-08-22 21:19:00+00:00   -1
2024-08-22 21:20:00+00:00   -1
2024-08-22 21:21:00+00:00   -1
2024-08-22 21:22:00+00:00   -1
2024-08-22 21:23:00+00:00   -1
                            ..
2024-08-23 03:32:00+00:00   -1
2024-08-23 03:33:00+00:00   -1
2024-08-23 03:34:00+00:00   -1
2024-08-23 03:35:00+00:00   -1
2024-08-23 03:36:00+00:00   -1
Name: position, Length: 378, dtype: int64

In [12]:
df_strategy['position'].iloc[-1] * 1000

np.int64(-1000)

In [13]:
ib.disconnect()